# Segmenting and Clustering Neighborhoods in Toronto Assignment

## 1. First Importing Librarys

### 1.1 Installing Required Library's  
Please if you haven't run the following code to install packages the remove the comments and run the code.

In [20]:
#! pip install geocoder                              #For Installing GeoCoder Library
#! conda install -c conda-forge geopy --yes          #For Installing Geopy Library
#! conda install -c conda-forge folium=0.5.0 --yes   #For Installing Folium

### 1.2 Importing library's

In [80]:
import requests        #For HTTP Requests
import lxml.html as lh #For Html to ? Convertion
import pandas as pd    #For Creating Dataframs
from lxml import etree #For Converting HTML Elements to Raw HTML
import numpy as np     #For Vectorized Computation
import geocoder        #For Getting the latitude and longitude Coordinates
from geopy.geocoders import Nominatim #For Converting an Address into latitude and longitude Values
import folium          #For Map Rendering
import json            #For handleing JSON files
from pandas.io.json import json_normalize #For Tranforming JSON File Into a Pandas Dataframe
from sklearn.cluster import KMeans  #For import K-means From Clustering Stage
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

## Second Preparing Data

Getting a data from a table located somewhere in a web page isn't that difficult, In order to get data then there is couple of things you need to know about HTML.
First HTML is a markup language which means it's written as tags like: 

and those tags always have a start and end so they look like something like this:  

The other thing is these tags sometimes contains attributes that defines some property's for the tag, A tag with an attribute will look like something like this:

There is two attributes that we need to focus on:
<ul>
    <li><strong>Class</strong> : Which defines the style class of the tag (Same Class Can Be Used For Multiple tags)</li>
    <li><strong>Id</strong>    : Which defines a unique name for the tag (Ids are Unique and Can't Be Used For Multiple tags)</li>
</ul>
So in order to get a tag then the first thing that you need to consider is to find the id of the tag (<font style='color:#f90909 !important;'>IF EXISTED</font>), and in order to get that tag is by opening your browser and 'Right Click' on that tag and click on 'Inspect' as follow:

<img src="Ex1.png" />

After that a small screen will appear and it will contain the HTML of the page and focus on the tag that you have 'Right-Clicked' on it, you will need to look for a 'Table' tag and try to find if there an id attribute with it.

<img src="Ex2.png" />

Based on the above image you can see that there is't an id with the table 'BUT' there is a class attribute so we will simply copy the value of that class and  assign it to a new variable in our notebook

In [22]:
Table_Class = "wikitable sortable" #Just Copy the First Two Words

### Data Enquiring
Now we will get the contents of the page and dig for that Table and for that we will gonna use LXML library

In [23]:
wikipedia_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'  #Page Url
page = requests.get(wikipedia_url)                                               #Get the Page
doc = lh.fromstring(page.content)                                                #getting the contents of the website

#Now we will get the wikipedia table that has a class attribute thet contints "wikitable sortable"
results  = doc.xpath("//table[@class='%s']" % Table_Class )

table = results[0]          #Getting The Table Which Should be the First in List (as HTML Element)
raw = etree.tostring(table) #Getting Raw HTML From HTML Element
df = pd.read_html(raw)[0]   #Use Pandas 'read_html' Function to Convert From HTML to Dataframe

df.head()                   #Printing Result Dataframe

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Cleaning Data
First we will delete every row that has 'Borough' value is not assign, then we will copy the value of 'Borough' column to 'Neighbourhood' that has no value and after that we will reset the dataframe index.
Next is kind of tricky; we will group data based on 'Postcode' and aggregate the child value into one cell but separate them by a comma  

In [24]:
df.drop(df.loc[df['Borough'] == "Not assigned"].index, inplace=True)         #Removing Rows That Has "Not assigned" Value
df.loc[df['Neighbourhood'] == "Not assigned" , 'Neighbourhood'] = df['Borough'];
df = df.reset_index(drop=True)                                               #Reseting Index
tmp = df.groupby(['Postcode'])['Neighbourhood'].agg(', '.join).reset_index() #Grouping Data
tmp['Borough'] = df.groupby(['Postcode'])['Borough'].first().reset_index()['Borough']
df = tmp                                 
df.shape

(103, 3)

### Preparing Latitude and Longitude 

In [25]:
lldf = pd.read_csv('Geospatial_Coordinates.csv')
df['Latitude'] = lldf.loc[lldf['Postal Code'] == df['Postcode'] , 'Latitude']
df['Longitude'] = lldf.loc[lldf['Postal Code'] == df['Postcode'] , 'Longitude']

df.head()

,Postcode,Neighbourhood,Borough,Latitude,Longitude
0,M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.763573,-79.188711
3,M1G,Woburn,Scarborough,43.770992,-79.216917
4,M1H,Cedarbrae,Scarborough,43.773136,-79.239476


In [26]:
lldf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Now we will use geopy library to get the latitude and longitude values of Toronto City.

<Strong>PS:</Strong> In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>Explorer</em>, as shown below.

In [27]:
address = 'Toronto City'

geolocator = Nominatim(user_agent="Explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7394839, -79.369314.


#### Create a map of Toronto with neighborhoods superimposed on top.  
For creating this map we will use Folium Library which is a cool open source library for creating geo-maps

In [30]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for _latitude, _longitude, _borough, _neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(_neighborhood, _borough)
    label = folium.Popup(label, parse_html=True)
    
                                #In Order To Create a Circle Marker Like the One on Google Then We'll Use the Folium Object 'CircleMarker'
    folium.CircleMarker(        #And Add The Following Parameters: -
        [_latitude, _longitude],                #1- Latitude And Longitude
        radius=5,                               #2- The Radius of the Circle
        popup=label,                            #3- The Lable That Will Pop Up
        color='blue',                           #4- The Color of the Circle
        fill=True,                              #5- Set Fill to True In Order to Fill the Circle
        fill_color='#3186cc',                   #6- This Parameter Represents the Filling Color
        fill_opacity=0.7,                       #7- The Level of Filling Opacity (Value Between 0 and 1)
        parse_html=False,                       #8-
        ).add_to(map_toronto)                   #Lastly We Assign the Marker to Our Map
    
map_toronto

### Utilizing Foursquare API

We are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.  
If you haven't created account in Foursquare then Click on the Link Bellow and Sign-up for a free account: -  
<a href="developer.foursquare.com"> Foursquare Sign-up </a>  
After that create a new app, fill in your personal information and click on create, after that click on 'Personal' Package and then down at the bottom click on (Return to My Apps).  
You should now see a card that contain the following informations: -  
<ul>
    <li>Client ID</li>
    <li>Client Secret</li>
<ul>

#### Define Foursquare Credentials and Version

In [42]:
CLIENT_ID = 'JYOC5WGRXKGGJN1LVBVVHA5RHQHCGSENQK2BCKKVNUQMHA2N'         # your Foursquare ID
CLIENT_SECRET = '0PI04RLZDG1SLY2DB2UV0ON45VHOH2QPDSDC00HPHESMRDYQ'     # your Foursquare Secret
VERSION = '20180605'                                                   # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JYOC5WGRXKGGJN1LVBVVHA5RHQHCGSENQK2BCKKVNUQMHA2N
CLIENT_SECRET:0PI04RLZDG1SLY2DB2UV0ON45VHOH2QPDSDC00HPHESMRDYQ


We'll use  a 5000 raduis and limit results to just 100

In [96]:
df.loc[0, 'Neighbourhood']
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighbourhood'] # neighborhood name
radius = 5000
LIMIT = 100
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


In [97]:
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=JYOC5WGRXKGGJN1LVBVVHA5RHQHCGSENQK2BCKKVNUQMHA2N&client_secret=0PI04RLZDG1SLY2DB2UV0ON45VHOH2QPDSDC00HPHESMRDYQ&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=5000&limit=100'

In [ ]:
results = requests.get(url).json()
results

In [50]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [101]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Toronto Pan Am Sports Centre,Athletics & Sports,43.790623,-79.193869
1,African Rainforest Pavilion,Zoo Exhibit,43.817725,-79.183433
2,Toronto Zoo,Zoo,43.820582,-79.181551
3,Polar Bear Exhibit,Zoo,43.823372,-79.185145
4,Australasia Pavillion,Zoo Exhibit,43.822563,-79.183286


In [102]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 3. Analyze Each Neighborhood

In [140]:
# one hot encoding
manhattan_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighbourhood'] = df['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()
#Saving Tmp
#manhattan_onehot.to_csv('Data1.csv')

,Neighbourhood,Athletics & Sports,Bakery,Bank,Beer Store,Breakfast Spot,Burger Joint,Campground,Caribbean Restaurant,Chinese Restaurant,...,Pizza Place,Pub,Sandwich Place,Smoothie Shop,Spa,Sri Lankan Restaurant,Steakhouse,Supermarket,Zoo,Zoo Exhibit
0,"Rouge, Malvern",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,Cedarbrae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [141]:
manhattan_onehot.shape

(100, 43)

In [142]:
manhattan_grouped = manhattan_onehot.groupby('Neighbourhood').mean().reset_index()
manhattan_grouped

,Neighbourhood,Athletics & Sports,Bakery,Bank,Beer Store,Breakfast Spot,Burger Joint,Campground,Caribbean Restaurant,Chinese Restaurant,...,Pizza Place,Pub,Sandwich Place,Smoothie Shop,Spa,Sri Lankan Restaurant,Steakhouse,Supermarket,Zoo,Zoo Exhibit
0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Alderwood, Long Branch",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bathurst Manor, Downsview North, Wilson Heights",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,Bayview Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Birch Cliff, Cliffside West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,"Bloordale Gardens, Eringate, Markland Wood, Ol...",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [143]:
manhattan_grouped.shape

(100, 43)

In [ ]:
num_top_venues = 5

for hood in manhattan_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [145]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [146]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = manhattan_grouped['Neighbourhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(1000)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Fried Chicken Joint,Zoo Exhibit,Coffee Shop,Grocery Store,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop,Cosmetics Shop
1,Agincourt,Park,Zoo Exhibit,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop,Cosmetics Shop,Coffee Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Campground,Zoo Exhibit,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop
3,"Alderwood, Long Branch",Bank,Zoo Exhibit,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop
4,"Bathurst Manor, Downsview North, Wilson Heights",Steakhouse,Zoo Exhibit,Chocolate Shop,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop,Cosmetics Shop
5,Bayview Village,Fried Chicken Joint,Zoo Exhibit,Coffee Shop,Grocery Store,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop,Cosmetics Shop
6,"Bedford Park, Lawrence Manor East",Fast Food Restaurant,Zoo Exhibit,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Discount Store,Diner,Dessert Shop,Cosmetics Shop
7,Berczy Park,Ice Cream Shop,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop,Cosmetics Shop
8,"Birch Cliff, Cliffside West",Zoo Exhibit,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop,Cosmetics Shop
9,"Bloordale Gardens, Eringate, Markland Wood, Ol...",Sandwich Place,Zoo Exhibit,Chocolate Shop,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop,Cosmetics Shop


## 4. Cluster Neighborhoods

In [147]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 3, 4], dtype=int32)

In [150]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

manhattan_merged = manhattan_merged[0:99]
manhattan_merged.head(1000) # check the last columns!

,Postcode,Neighbourhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353,0.0,Athletics & Sports,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop,Cosmetics Shop
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,43.784535,-79.160497,3.0,Zoo Exhibit,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop,Cosmetics Shop
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.763573,-79.188711,0.0,Zoo,Zoo Exhibit,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop
3,M1G,Woburn,Scarborough,43.770992,-79.216917,0.0,Zoo,Zoo Exhibit,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop
4,M1H,Cedarbrae,Scarborough,43.773136,-79.239476,3.0,Zoo Exhibit,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop,Cosmetics Shop
5,M1J,Scarborough Village,Scarborough,43.744734,-79.239476,3.0,Zoo Exhibit,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop,Cosmetics Shop
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough,43.727929,-79.262029,0.0,Bakery,Zoo Exhibit,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough,43.711112,-79.284577,3.0,Zoo Exhibit,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop,Cosmetics Shop
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough,43.716316,-79.239476,3.0,Zoo Exhibit,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop,Cosmetics Shop
9,M1N,"Birch Cliff, Cliffside West",Scarborough,43.692657,-79.264848,3.0,Zoo Exhibit,Coffee Shop,Grocery Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant,Discount Store,Diner,Dessert Shop,Cosmetics Shop


In [151]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
 
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighbourhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color= rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters